# 📈 E-commerce Sales Analysis Portfólio

Este notebook apresenta uma análise completa de dados de vendas de um e-commerce fictício. O objetivo é demonstrar habilidades em **Extração, Transformação e Carga (ETL)**, **Análise Exploratória de Dados (EDA)** e **Visualização Interativa** utilizando as bibliotecas `Pandas` e `Plotly`.

## 🛠️ Tecnologias Utilizadas
- **Python 3.10+**
- **Pandas**: Manipulação e análise de dados.
- **Plotly**: Gráficos interativos de última geração.
- **Numpy**: Operações matemáticas.

## 1. ⚙️ Setup e Importação de Dados

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Carregando o dataset gerado
df = pd.read_csv('ecommerce_sales.csv', parse_dates=['OrderDate'])

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
df.head()

## 2. 🧹 Data Cleaning e Feature Engineering

Nesta etapa, vamos garantir que os tipos de dados estejam corretos e criar colunas úteis para a análise temporal.

In [ ]:
# Criando colunas de Ano/Mês para análise temporal
df['YearMonth'] = df['OrderDate'].dt.to_period('M').astype(str)
df['DayOfWeek'] = df['OrderDate'].dt.day_name()

# Verificando valores nulos
print("Valores nulos por coluna:\n", df.isnull().sum())

df.info()

## 3. 📊 Análise Exploratória (EDA)

### 3.1. Visão Geral das Vendas
Total de receita, quantidade de pedidos e ticket médio.

In [ ]:
total_revenue = df['TotalPrice'].sum()
total_orders = df['OrderID'].nunique()
average_ticket = total_revenue / total_orders

print(f"Receita Total: R$ {total_revenue:,.2f}")
print(f"Total de Pedidos: {total_orders}")
print(f"Ticket Médio: R$ {average_ticket:,.2f}")

### 3.2. Tendência Mensal de Faturamento
Vamos visualizar como a receita se comportou ao longo do tempo.

In [ ]:
monthly_sales = df.groupby('YearMonth')['TotalPrice'].sum().reset_index()

fig_trend = px.line(monthly_sales, x='YearMonth', y='TotalPrice', 
                   title='Tendência Mensal de Faturamento', 
                   labels={'TotalPrice': 'Receita (R$)', 'YearMonth': 'Mês'},
                   markers=True,
                   template='plotly_dark')
fig_trend.show()

### 3.3. Performance por Categoria e Produto

In [ ]:
# Vendas por Categoria
category_sales = df.groupby('Category')['TotalPrice'].sum().sort_values(ascending=False).reset_index()

fig_cat = px.bar(category_sales, x='Category', y='TotalPrice', 
                title='Faturamento por Categoria', 
                color='TotalPrice',
                color_continuous_scale='Viridis',
                template='plotly_dark')
fig_cat.show()

### 3.4. Distribuição Geográfica (Top Países)

In [ ]:
country_sales = df.groupby('Country')['TotalPrice'].sum().reset_index()

fig_geo = px.choropleth(country_sales, 
                        locations='Country', 
                        locationmode='country names',
                        color='TotalPrice', 
                        hover_name='Country', 
                        title='Distribuição Global de Vendas',
                        color_continuous_scale='Plasma',
                        template='plotly_dark')
fig_geo.show()

## 4. 📝 Conclusões e Insights

Nesta seção, extraímos as principais métricas de negócio de forma dinâmica diretamente dos dados analisados.

In [ ]:
from IPython.display import Markdown

# Cálculos para Insights Dinâmicos
top_cat_row = category_sales.iloc[0]
top_cat_name = top_cat_row['Category']
top_cat_perc = (top_cat_row['TotalPrice'] / total_revenue) * 100

top_country_name = country_sales.sort_values('TotalPrice', ascending=False).iloc[0]['Country']

insights = f"""
1. **Identificação de Sazonalidade**: O gráfico de tendência temporal permite identificar os períodos de pico de faturamento.
2. **Dominância de Categoria**: A categoria **{top_cat_name}** é a líder de vendas, representando **{top_cat_perc:.1f}%** do faturamento total.
3. **Geografia**: O mercado de **{top_country_name}** se destaca como o país com maior volume de vendas em valor monetário.

---
"""
display(Markdown(insights))